# Pre-need

In [1]:
import pandas as pd
from scipy.stats import entropy
import numpy as np
import collections
import pandas as pd
import time
import random
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo   
import math

def order(attri,i,datax):
    for j in range(len(datax[i])):
        if(attri == datax[i][j]):
            return j
        
def attri(x, i, datax):
    return datax[i][x]
def get_s(X, loca):
    s = []
    if(loca > X.shape[1]):
        return s
    else:
        for i in range(X.shape[1]):
            if(i>=loca):
                s.append(X.iloc[:, i].value_counts().index)
        for i in range(X.shape[1]-loca):
            s[i] = np.sort(s[i]) 
        return s  

# CADM

In [2]:
def weighted_mutual_infoa(cen, q, attr_idx, intra_dis, opt):

    gama_cen = intra_dis[attr_idx][cen]
    if(opt == True):
        gama_q = intra_dis[attr_idx][q]
    else:
        gama_q = 1/intra_dis[attr_idx][q]
        
    mi_pq = gama_cen + gama_q
    
    return  mi_pq

def pairwise_distance_ca2(X, Cen,k1,data,location,datax,importance_freq,intra_dis,opt):
    """
    :param X: sample
    :param Cen: centroid
    :return: the distance between two objects
    """
    n = X.shape[1]
    distance = 0
    
    for i in range(n):
        p = X[:,i]
        qc = Cen[:,i]
        if(i >=location):
            if(p == qc):
                distance += 0
            else:
                rank1 = order(p, i-location,datax) #order function 
                rankc = order(qc, i-location, datax)
                diff = abs(rank1-rankc) 
                first = np.min([rank1, rankc])  
                gama_cen = intra_dis[i][qc[0]]
                self = False

                for j in range(0,diff+1):
                    loc1 = first + j
                    it = attri(loc1, i-location, datax) #attri return attribut
                    
                    if(it != qc[0]):
                        mutual_info = weighted_mutual_infoa(qc[0],it, i, intra_dis, opt)
                        distance += mutual_info
#                 distance = distance/(diff)
                distance += importance_freq[i]*importance_freq[i]
                
        else:
            if(p == qc):
                distance += 0
            else:
                self = True

                mutual_info = weighted_mutual_infoa(qc[0],p[0], i, intra_dis, opt)
                distance += mutual_info + importance_freq[i]*importance_freq[i]
            
    return distance        
            
def cluster_data_ca2(rs,df, location, datax, k1):   
 # k centers
    k = k1
    centroids = df.sample(k, random_state=rs).to_numpy()#1647
#     centroids = df.iloc[[0, 2, 4]].to_numpy()
    prev_centroids = centroids.copy()
    
    importance_freq = []
    intra_dis = []
    convergence = []
    
        
    for i1 in range(k):
        importance_freq.append({})
        intra_dis.append([])
        for j in range(df.shape[1]):
            intra_dis[i1].append({})
                       
                            
    labels = np.zeros(df.shape[0], dtype=int)
    cns = df.shape[0]
    
    min_distance = [float('inf')] * df.shape[0]
    opt = False
#     print(corr_vec)
    for turn in range(100):
        stable = True
#         print(f"Iteration {turn}")
        if(turn == 0):
            opt = True
        else:
            opt = False
        
        for j in range(k):
            if(turn > 0):
                cluster_points = df[labels == j]
            else:
                cluster_points = df
                    
            for i in range(cluster_points.shape[1]):
                counter = collections.Counter(cluster_points.iloc[:, i])
                most_common_times = counter.most_common(1)[0][1]
                for j1 in cluster_points.iloc[:, i].value_counts().index:
                    intra_dis[j][i][j1] = cluster_points.iloc[:, i].value_counts()[j1]/cns

            s_t = 0
            for i1 in range(cluster_points.shape[1]):
                 #importance
                counter = collections.Counter(cluster_points.iloc[:, i1])
                current_common_times = counter.most_common(1)[0][1]
                s_t += current_common_times
            for i1 in range(cluster_points.shape[1]):
                counter = collections.Counter(cluster_points.iloc[:, i1])
                current_common_times = counter.most_common(1)[0][1]   
                importance_freq[j][i1] =  current_common_times/s_t
                
        for i in range(df.shape[0]):
            for j in range(k):
                if(turn > 0):
                    cluster_points = df[labels == j]
                else:
                    cluster_points = df
                distance = pairwise_distance_ca2(df.iloc[i:i+1].to_numpy(), centroids[j:j+1], k1,df,location, datax,importance_freq[j], intra_dis[j],opt)
                if distance < min_distance[i]:
                    min_distance[i] = distance
                    labels[i] = j
                    stable = False

        for j in range(k):
            cluster_points = df[labels == j]
            if cluster_points.shape[0] > 0:
                new_centroids = []
                for col in df.columns:
                    counter = collections.Counter(cluster_points[col])
                    mode = counter.most_common(1)[0][0]
                    new_centroids.append(mode)
                centroids[j] = np.array(new_centroids)
                
         
        if stable:
            
            break

    return labels, centroids


# NMI

In [3]:
from sklearn.metrics import mutual_info_score
def nmi(y,cluster_labels,real): 
#     for i in range(len(real)):
#         y.loc[y['class'] == real[i], 'class'] = i
    y = y.squeeze() 
    for i in range(len(real)):
        for j in range(len(real)):
            if(cluster_labels[i] == j):
                cluster_labels[i] = real[j]

    mi = mutual_info_score(y, cluster_labels)

    h_y = entropy(np.unique(y, return_counts=True)[1] / len(y))
    h_y_pred = entropy(np.unique(cluster_labels, return_counts=True)[1] / len(cluster_labels))

    nmi = mi / max(h_y, h_y_pred)

    print(f"NMI: {nmi:.3f}")
    return nmi

In [4]:
def nm_vo(y, cluster_labels):
    y = y.squeeze() 
    vi1s = mutual_info_score(y, cluster_labels)
    h_y = entropy(np.unique(y, return_counts=True)[1] / len(y))
    h_y_pred = entropy(np.unique(cluster_labels, return_counts=True)[1] / len(cluster_labels))
    return vi1s / max(h_y, h_y_pred) 

# CA

In [5]:
def cal(X,y,cluster_labels):
    a1 = []
    z = []
    real = []
    y = y.reset_index(drop=True)
    for i in range(len(y.value_counts())):
        a = []
        for i1 in range(X.shape[0]):
            if(cluster_labels[i1] == i):
                a.append(i1)
        cnt = 0
        z1 = 0
        rea = 0
        for j in range(len(y.value_counts())):
            lo = y.value_counts().index[j][0]
            one_index = y[y.iloc[:,0] == lo].index
            cnt1 = 0
            for i in a:
                for j in one_index:
                    if(i == j):
                        cnt1 += 1
            if(j == 0):
                cnt = cnt1
                rea = lo
            else:
                if(cnt<cnt1):
                    s1 = False
                    for j1 in z:
                        if(j == j1):
                            s1 = True
                    if(s1 == False):
                        cnt = cnt1
                        z1 = j
                        rea = lo

        a1.append(cnt)
        z.append(z1)
        real.append(rea)
        
    cx = 0
    for i in a1:
        cx+=i
    cs = cx/X.shape[0]
    return cs,real

# ARI

In [6]:
from sklearn.metrics.cluster import adjusted_rand_score
def ARI(cluster_labels,x):
    x = x.squeeze()
    ari = adjusted_rand_score(cluster_labels, x)
    print(f"Adjusted Rand Index (ARI): {ari:.3f}")
    return ari

# Hayes (HA)

In [16]:
# fetch dataset 

hayes_roth = fetch_ucirepo(id=44) 
  
# data (as pandas dataframes) 

X = hayes_roth.data.features 
y = hayes_roth.data.targets 
# delete null
X = X.loc[~y.isnull().any(axis=1)]
y = y.loc[~y.isnull().any(axis=1)]

In [ ]:
od = 2
s = 0
nm=0
a=0
for i in range(10):
    cluster_labels, centroids = cluster_data_ca2(44,X,od,get_s(X, od),3)
    cs, real = cal(X,y,cluster_labels)
    hai1 =ARI(cluster_labels,y)
    hi1 = nmi(y,cluster_labels,real)
    s+=cs
    a+=hai1
    nm+=hi1 
print(f'Accuracy: {s/10}')
print(f'NMI: {nm/10}')
print(f'ARI: {a/10}')

# LY

In [19]:
from ucimlrepo import fetch_ucirepo 
lymphography = fetch_ucirepo(id=63) 
  
    
# data (as pandas dataframes) 
X = lymphography.data.features 
y = lymphography.data.targets 
X = X.drop(X.columns[2], axis=1)
X = X.drop(X.columns[-1], axis=1)
X = X.drop(X.columns[11], axis=1)

In [ ]:
od = 19
s = 0
nm=0
a=0
for i in range(10):
    cluster_labels, centroids = cluster_data_ca2(91,X,od,get_s(X, od),4)
    cs, real = cal(X,y,cluster_labels)
    hai1 =ARI(cluster_labels,y)
    hi1 = nmi(y,cluster_labels,real)
    s+=cs
    a+=hai1
    nm+=hi1    
print(f'Accuracy: {s/10}')
print(f'NMI: {nm/10}')
print(f'ARI: {a/10}')

# Mass(MA)

In [10]:
# fetch dataset 
mammographic_mass = fetch_ucirepo(id=161) 
  
# data (as pandas dataframes) 
X = mammographic_mass.data.features 
y = mammographic_mass.data.targets 
X = X.dropna()
y = y.loc[X.index]
X = X.drop(columns=['Age'])
X = X[list(X.columns[1:3]) + [X.columns[0]] + [X.columns[3]]]

In [ ]:
od = 2
s = 0
nm=0
a=0
for i in range(10):
    cluster_labels, centroids = cluster_data_ca2(59,X,od,get_s(X, od),2)
    cs, real = cal(X,y,cluster_labels)
    hai1 =ARI(cluster_labels,y)
    hi1 = nmi(y,cluster_labels,real)
    s+=cs
    a+=hai1
    nm+=hi1    
print(f'Accuracy: {s/10}')
print(f'NMI: {nm/10}')
print(f'ARI: {a/10}')

# Primary(PR)

In [24]:
# fetch dataset 
primary_tumor = fetch_ucirepo(id=83) 
  
# data (as pandas dataframes) 
X = primary_tumor.data.features 
y = primary_tumor.data.targets 

X = X.dropna()
y = y.loc[X.index]
d = y.value_counts()[y.value_counts() >= 9].index
index = []
for i in d:
    index.extend(y.loc[y.iloc[:,0] == i[0], :].index)
X = X.loc[index]
y = y.loc[index]

In [ ]:
od = 20
s = 0
nm=0
a=0
import numpy as np
from sklearn.metrics import normalized_mutual_info_score
for i in range(10):
    cluster_labels, centroids = cluster_data_ca2(46,X,od,get_s(X, od),5)
    cs, real = cal(X,y,cluster_labels)
    hai1 =ARI(cluster_labels,y)
    hi1 = nmi(y,cluster_labels,real)
    s+=cs
    a+=hai1
    nm+= hi1 
print(f'Accuracy: {s/10}')
print(f'NMI: {nm/10}')
print(f'ARI: {a/10}')

# Nursery(NS)

In [11]:
nursery = fetch_ucirepo(id=76) 
  
# data (as pandas dataframes) 
X = nursery.data.features 
y = nursery.data.targets 
datax = [['usual', 'pretentious', 'great_pret'],
 ['proper', 'less_proper', 'improper', 'critical', 'very_crit'],
 ['complete', 'completed', 'incomplete', 'foster'],
 ['1', '2', '3', 'more'],
 ['convenient', 'less_conv', 'critical'],
 ['inconv', 'convenient'],
 ['nonprob', 'slightly_prob', 'problematic'],
 ['not_recom', 'recommended', 'priority']]

In [ ]:
od = 0
cluster_labels, centroids = cluster_data_ca2(17,X,od,datax,4)
cs, real = cal(X,y,cluster_labels)
hai1 =ARI(cluster_labels,y)
# y = y.squeeze() 
# ni1s = mutual_info_score(cluster_labels,y)
# h_y = entropy(np.unique(y, return_counts=True)[1] / len(y))
# h_y_pred = entropy(np.unique(cluster_labels, return_counts=True)[1] / len(cluster_labels))
# ni1 = ni1s / max(h_y, h_y_pred)
ni1 = nm_vo(y, cluster_labels)
print(f'Accuracy: {cs}')
print(f'NMI: {ni1}')

# Lenses(LEN)

In [26]:
# fetch dataset 
lenses = fetch_ucirepo(id=58) 
  
# # data (as pandas dataframes) 
X = lenses.data.features 
y = lenses.data.targets 

In [ ]:
od = 4
s = 0
nm=0
a=0
for i in range(10):
    cluster_labels, centroids = cluster_data_ca2(28,X,od,get_s(X, od),3)
    cs, real = cal(X,y,cluster_labels)
    hai1 =ARI(cluster_labels,y)
    hi1 = nmi(y,cluster_labels,real)
    s+=cs
    a+=hai1
    nm+=hi1    
print(f'Accuracy: {s/10}')
print(f'NMI: {nm/10}')
print(f'ARI: {a/10}')

# Voting(VO)

In [15]:
# fetch dataset 
congressional_voting_records = fetch_ucirepo(id=105) 
  
# data (as pandas dataframes) 
X = congressional_voting_records.data.features 
y = congressional_voting_records.data.targets 
X = X.dropna()
y = y.loc[X.index]

In [ ]:
od = 17
s = 0
nm=0
a=0
for i in range(10):
    cluster_labels, centroids = cluster_data_ca2(72,X,od,get_s(X, od),2)
    cs, real = cal(X,y,cluster_labels)
    hai1 =ARI(cluster_labels,y)
    s+=cs
    a+=hai1
    nm+= nm_vo(y, cluster_labels)
print(f'Accuracy: {s/10}')
print(f'NMI: {nm/10}')
print(f'ARI: {a/10}')